# Seldon V2 Kubernetes Examples

 * Build if needed and place `seldon` binary in your path
 * Install Seldon on Kubernetes


In [ ]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP
MESH_IP

In [ ]:
!which seldon

## Model

In [ ]:
!cat ./models/sklearn-iris-gs.yaml

In [ ]:
!kubectl create -f ./models/sklearn-iris-gs.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!kubectl get model iris -n seldon-mesh -o jsonpath='{.status}' | jq .

In [ ]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!kubectl get server mlserver -n seldon-mesh -o jsonpath='{.status}' | jq .

In [ ]:
!kubectl delete -f ./models/sklearn-iris-gs.yaml

## Experiment

In [ ]:
!cat ./experiments/sklearn1.yaml

In [ ]:
!cat ./experiments/sklearn2.yaml 

In [ ]:
!kubectl create -f ./experiments/sklearn1.yaml
!kubectl create -f ./experiments/sklearn2.yaml

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!cat ./experiments/ab-default-model.yaml 

In [ ]:
!kubectl create -f ./experiments/ab-default-model.yaml 

In [ ]:
!kubectl wait --for condition=ready --timeout=300s experiment --all -n seldon-mesh

In [ ]:
!seldon model infer --inference-host ${MESH_IP} --inference-port 80 -i 50 --model-name iris \
  '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}' 

In [ ]:
!kubectl delete -f ./experiments/ab-default-model.yaml 
!kubectl delete -f ./experiments/sklearn1.yaml
!kubectl delete -f ./experiments/sklearn2.yaml

## Pipeline (WIP)

In [ ]:
!cat ./pipelines/example.yaml

In [ ]:
!kubectl create -f ./pipelines/example.yaml

In [ ]:
!kubectl delete -f ./pipelines/example.yaml